<a href='http://www.holoviews.org'><img src="../../assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Exercise 3: Networks and GeoViews</h2></div>

In [ ]:
import numpy as np  # noqa
import pandas as pd
import holoviews as hv
from holoviews import opts, dim  # noqa
import geoviews as gv
import networkx as nx

from bokeh.sampledata.airport_routes import airports, routes

hv.extension('bokeh')

Note that if you see an error about ``airports.csv`` not existing when running the above line, make sure to run the *Downloading sample data* section of the [Setup notebook](../00_Setup.ipynb).


### Example 1

In this exercise we will plot the locations of all US airports on a map. Before we get started let's inspect the ``airports`` dataframe using the ``head`` method.

Now declare a ``gv.Points`` element from the ``airports`` dataframe plotting the 'Longitude' and 'Latitude', assign the object to a variable, and display it. Be sure to think carefully about what kind of dimensions (key or value) those columns represent.

<details><summary href="#hint1">Hint</summary>

``Points``, unlike ``Scatter``, has two key dimensions (``kdims``). It represents a two-dimensional space (in this case the position of each airport on a map).
</details>

<details><summary href="#solution1">Solution</summary>
<br>

```python
points = gv.Points(airports, ['Longitude', 'Latitude'])
points
```

<br>
</details>

You should now be able to see that the dataset includes various US airbases in Europe, in addition to those actually located in US states. Let's focus only on airports in US states by using the ``select`` method to select airports between -180 and 0 in Longitude and above 0 degrees in Latitude, and assign that to your variable.

<details><summary href="#hint2">Hint</summary>

The select method allows selecting coordinates, multiple values and ranges of values, e.g. to select rows where the 'value' column has values between 0 and 100 use ``dataset.select(column=(0, 100))``.  ``None`` can be used for any range start or end that you do not need to enforce.
    
</details>

<details><summary href="#solution2">Solution</summary>
<br>

```python
points = points.select(Longitude=(-180, 0), Latitude=(0, None))
points
```

<br>
</details>

Finally, overlay the points on a map tile source using the ``gv.WMTS`` element and the tile source URL provided below. Then adjust the width and height of the ``Points`` and enable the 'hover' tool.

<details><summary href="#hint3">Hint</summary>

Tools can be enabled on bokeh plots by supplying a list of tools, e.g. ``tools=['box_select', 'hover']``.
</details>

In [ ]:
url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}.png'

<details><summary href="#solution3">Solution</summary>

<br>

```python
gv.WMTS(url) * points.opts(width=500, height=400, tools=['hover'], size=2, color='black', fill_alpha=0)
```

</details>

### Example 2
In this exercise we will plot a network graph of all the airport connections. As always, first inspect the relevant datasets. We've already seen the structure of the ``airports`` dataset, so let's look at the ``routes`` dataframe as well:

Let's make a NetworkX graph to work with this data about airport routes:

In [ ]:
g = nx.from_pandas_edgelist(routes, 'SourceID', 'DestinationID')

``g`` is an abstract graph object, but we can make a visualizable ``hv.Graph`` object from it using the ``hv.Graph.from_networkx`` classmethod. ``Graph.from_networkx`` accepts the NetworkX graph as the first argument and a layout function such as ``nx.spring_layout`` as the second argument. Once it displays, reduce the ``node_size`` so you can make out the different nodes.

<details><summary href="#hint4">Hint</summary>

``Graph`` style options are split into ``node_`` and ``edge_`` options.
</details>

<details><summary href="#solution4">Solution</summary><br>

```python
hv.Graph.from_networkx(g, nx.spring_layout).opts(node_size=4)
```

<br></details>

If you hover over the nodes you will notice that while it includes an index the hover information is otherwise not very useful. We will now add additional node information by supplying a Dataset indexed by the AirportID. 

First, declare a ``hv.Dataset`` for the ``airports`` with the 'AirportID' as a key dimension and the 'Name' and 'TZ' (or timezone) as value dimensions. Then supply the Dataset as the third argument to the ``Graph.from_networkx`` function and check that the hover now shows more useful information.

Finally, customize the plot by adjusting ``width`` and ``height``, adjusting the ``node_size`` and ``edge_line_width`` and add a ``color_index`` and a ``cmap`` of 'tab20'.

<details><summary href="#hint5">Hint</summary>

Ensure the ``Dataset`` with additional node info defines 'AirportID' as the sole key dimension. 
</details>

<details><summary href="#solution5">Solution</summary>

<br>

```python
graph_opts = opts.Graph(width=600, height=600, color=dim('TZ'), node_size=4, cmap='tab20', edge_line_width=1)
ds = hv.Dataset(airports, 'AirportID', ['Name', 'TZ'])
hv.Graph.from_networkx(g, nx.spring_layout, ds).opts(graph_opts)
```

</details>

### Example 3

In this exercise we will combine what we learned about geographic data with graph support to visualize the flight connections on a map. Since there are many more connections than we can easily view, we will count the number of connections between airports and select the fifty busiest.

In [ ]:
# Count the number of connections from each airport
counts = routes.groupby('SourceID')[['Stops']].count().reset_index().rename(columns={'Stops': 'Connections'})
airports_df = pd.merge(airports, counts, left_on='AirportID', right_on='SourceID', how='left')

# Select only airports located in US states & convert from Web Mercator to Latitudes/Longitudes
airport_points = gv.Points(airports_df, ['Longitude', 'Latitude']).select(Longitude=(-180, 0), Latitude=(0, None))
projected_points = gv.operation.project_points(airport_points)

busiest_ids = list(routes.groupby('SourceID').count().sort_values('Stops').iloc[-50:].index.values)
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

We now have the ``AirportIDs`` for the 50 busiest airports defined on the ``busiest_ids`` variable, a tile source defined on the ``tiles`` variable and below we have already defined a number of options and a ``Nodes`` element.

Define an hv.Graph from the ``routes`` and ``nodes``, declaring 'SourceID' and 'DestinationID' as the key dimensions. Then use the ``graph.select`` method to select by ``AirportID`` using the ``busiest_ids``. Then overlay the selected Graph on top of the ``tiles``. Also try switching the ``selection_mode`` on the ``select`` method from 'edges' to 'nodes' and observe the difference.

<details><summary href="#hint6">Hints</summary>

When constructing a ``Graph`` supply the edges and nodes as tuple, e.g. ``hv.Graph((edges, nodes))``.
    
The ``selection_mode`` is a special keyword of the Graph.select method.
</details>

In [ ]:
graph_opts = opts.Graph(width=800, height=800, edge_selection_line_color='black', edge_hover_line_color='red',
                        node_size=8, edge_line_width=1, edge_line_alpha=0, edge_nonselection_line_alpha=0)
nodes = hv.Nodes(projected_points, ['Longitude', 'Latitude', 'AirportID'], 
                                   ['Name', 'City', 'Connections']).opts(graph_opts)



<details><summary href="#solution6">Solution</summary><br>
    
```python
graph_opts = opts.Graph(width=800 height=800, edge_selection_line_color='black', edge_hover_line_color='red',
                        node_size=8, edge_line_width=1, edge_line_alpha=0, edge_nonselection_line_alpha=0)
nodes = hv.Nodes(projected_points, ['Longitude', 'Latitude', 'AirportID'], 
                                   ['Name', 'City', 'Connections']).opts(graph_opts)
```

<br>
    
**Declare nodes, graph and tiles**

```python
graph = hv.Graph((routes, nodes), ['SourceID', 'DestinationID'])
```
    
<br>
    
**Select 50 busiest airports**


```python
busiest_airports = graph.select(AirportID=busiest_ids, selection_mode='edges')
tiles * busiest_airports
```

<br></details>